# Notebook to invesitgate why solution gets "stuck"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nodepy import rk
import cvxpy as cp


import numpy.linalg as linalg


rk4 = rk.loadRKM('RK44').__num__()
rk4x2 = rk4*rk4
ssp2 = rk.loadRKM('SSP22').__num__()
ssp3 = rk.loadRKM('SSP33').__num__()
ssp104 = rk.loadRKM('SSP104').__num__()
merson4 = rk.loadRKM('Merson43').__num__()
bs5 = rk.loadRKM('BS5').__num__()

trbdf = rk.loadRKM('TR-BDF2').__num__()
be = rk.loadRKM('BE').__num__()
irk2 = rk.loadRKM('LobattoIIIA2').__num__()

In [ ]:
def tau(k,A,c): 
    #generates tau vector 
    return 1/ np.math.factorial(k)*c**k - 1/np.math.factorial(k-1) *A @ c**(k-1)
    
def tau_hat(k,A,c):
    return c**k-k*A@(k-1)


def OrderCond(A,c,order = 1):
    #Generates Order Condition Matrix O and right side vector r for Linear Equation System O@b=r
    
    s = len(c) #number of stages
    
    r = []
    O_rows = []
    
    
    if A.shape != (s,s):
        raise InputError
        
    else:
        if order >= 1:
            O_rows.append(np.ones(s));      r.append(1)
            
        if order >= 2:
            O_rows.append(c);               r.append(1/2)
            
        if order >= 3:
            O_rows.append(c**2);            r.append(1/3)
            O_rows.append(tau(2,A,c));      r.append(0.)
            
        if order >= 4:
            O_rows.append(c**3);            r.append(1/4)
            O_rows.append(tau(2,A,c)*c);    r.append(0.)
            O_rows.append(tau(2,A,c)@A.T);  r.append(0.)
            O_rows.append(tau(3,A,c));      r.append(0.)
        
        if order >= 5:
            print('too high order')
            raise NotImplementedError
        
        O = np.vstack(O_rows)
        return (O,np.array(r))
            
                
    

In [ ]:
def RK_variable_b(rkm, dt, f, w0=[1.,0], t_final=1.,b_fixed = False):
    """    
    Options:
    
        rkm: Base Runge-Kutta method, in Nodepy format
        dt: time step size
        f: RHS of ODE system
        w0: Initial data
        t_final: final solution time        
    """
    
    #setup Variables for Soulution storage
    p = len(w0) #number of dimentions
    
    uu = np.zeros([p,int(t_final/dt)+100])
    uu[:,0] = w0.copy()
    tt = np.zeros([int(t_final/dt)+100])
    
    
    #Setup Runge Kutta 
    c = rkm.c
    A = rkm.A #has to be lower left triangle
    s = len(c) #number of Stages
    K = np.zeros([p,s])
    
    u = np.array(w0)
    t = 0.
    n = 0
    
    
    #Setup Optimisation Problem
    O, rhs = OrderCond(rkm.A,rkm.c,order = rkm.p)
    b_op =cp.Variable(s)
    e = np.ones(s) #vector for gola Fnction, just generates the 1-Norm of b
    
    #Maybee set up Problem here and treat H as an Paramter
        
    #for debugging b's    
    bb = np.zeros([s,int(t_final/dt)+2])
        
    #print('set up starting to solve')
    
    #Solve ODE
    while t<t_final:
        for i in range(s):
            #compute Stages
            
                
            #K[:,i] = f(t+c[i]*dt,u+dt*K@A[i,:]) 
            #the 0s in A should make shure that no data from an older Step is used
            
            #Maybe better Approach, because A[i,j] = 0 in many places
            u_prime = u.copy()
            for m in range(i):
                u_prime += dt*A[i,m]*K[:,m]
            
            K[:,i] = f(t+c[i]*dt,u_prime)
            
            #print('intermediatestep computed')
        
        if b_fixed == False:
            #Run Optimisation Problem
        
            prob = cp.Problem(cp.Minimize(e@b_op),[O@b_op==rhs,u+dt*K@b_op>=0])  
            prob.solve()
            if prob.status != cp.OPTIMAL:
                print(prob.status)
            
            b = b_op.value
        else:
            b =rkm.b
        #update
        u += dt*K@b
        n += 1
        t += dt
        
        uu[:,n] = u.copy()
        bb[:,n] = b.copy()
        tt[n] = t
        #print('updated')

        
    return (tt[0:n],uu[:,0:n],bb[:,0:n])
        
        
    

In [ ]:
#Testproblem from Kopecz and Meister 2018

def f_lin_I(t,u):
    a = 5 # a>0
    A = np.array([[-a,1],[a,-1]])
    return A@u

u0 =np.array([0.9,0.1])

t,u,b = RK_variable_b(ssp104,2,f_lin_I,w0=u0,t_final=100,b_fixed=False)




In [ ]:
plt.plot(t,u[0,:])
plt.plot(t,u[1,:])

In [ ]:
plt.plot(t,b.T)

## Problem statement

$ \Delta t(b_1 k_1 + \cdots + b_s k_s  ) = 0 $ for all steps
Obviously this is not correct

Why?
Which influence does objective Function have?
How can it be solved?


At first we make shure there is no error in the code and $b$ comlies with order conditions

In [ ]:
#setup Order conditions
rkm = ssp104
O, rhs = OrderCond(rkm.A,rkm.c,order = rkm.p)

#pick b from debug data
bb = b[:,8]

#test
err = O@bb -rhs

err

In [ ]:
u =np.array(u0)

## Extract optimisation Problem

In [ ]:
#computing the stages of a single Runge Kutta step

f = f_lin_I

dt =2
p = len(u0)

#Setup Runge Kutta 
c = rkm.c
A = rkm.A #has to be lower left triangle
s = len(c) #number of Stages
K = np.zeros([p,s])
    
#u = np.array(u0)
t = 0.
n = 0
    
    
#Setup Optimisation Problem
O, rhs = OrderCond(rkm.A,rkm.c,order = rkm.p)
b_op =cp.Variable(s)
e = np.ones(s) #vector for gola Fnction, just generates the 1-Norm of b




for i in range(s):

    u_prime = u.copy()
    for m in range(i):
        u_prime += dt*A[i,m]*K[:,m]  
    K[:,i] = f(t+c[i]*dt,u_prime)
      

#Running optimisaion Problem
            
prob = cp.Problem(cp.Minimize(e@b_op),[O@b_op==rhs,u+dt*K@b_op>=0])  
prob.solve()
if prob.status != cp.OPTIMAL:
    print(prob.status)
            
b = b_op.value
        
#update
u += dt*K@b
    
print(dt*K@b)

In [ ]:
print(u)

In [ ]:
print(b)

In [ ]:
print(O@b -rhs)

In [ ]:
print(O@rkm.b -rhs)